In [10]:
# Import necessary libraries
import pandas as pd
import numpy as np
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import ipywidgets as widgets
from IPython.display import display

In [11]:
# Load movie and ratings data
movies_df = pd.read_csv("~/Desktop/MovieLens-resources/movies.csv")

ratings_df = pd.read_csv("~/Desktop/MovieLens-resources/ratings.csv")
ratings_df = ratings_df.drop('timestamp',axis=1)

In [12]:
# Function to clean movie titles
def clean_title(title):
    title = re.sub("[^a-zA-Z0-9 ]", "", title)
    return title


In [13]:
# Apply title cleaning to the movies dataframe
movies_df["clean_title"] = movies_df["title"].apply(clean_title)

In [14]:
# Create TF-IDF vectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))

tfidf = vectorizer.fit_transform(movies_df["clean_title"])

In [15]:
# Function to search for movies based on input title
def search(title):
    title = clean_title(title)
    query_vec = vectorizer.transform([title])
    similarity = cosine_similarity(query_vec, tfidf).flatten()
    indices = np.argpartition(similarity, -5)[-5:]
    results = movies_df.iloc[indices].iloc[::-1]
    
    return results

In [16]:
# Widget for inputting movie title and displaying search results
movie_input = widgets.Text(
    
    description='Movie Title:',
    disabled=False
)
movie_list = widgets.Output()

# Callback function for handling text input changes
def on_type(data):
    with movie_list:
        movie_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            display(search(title))

movie_input.observe(on_type, names='value')


display(movie_input, movie_list)

Text(value='', description='Movie Title:')

Output()

In [17]:
def find_similar_movies(movie_id):
    # Find users who like the same movie as us
    similar_users = ratings_df[(ratings_df["movieId"] == movie_id) & (ratings_df["rating"] > 4)]["userId"].unique()
    # Find any movie that similar_users rated 5 star
    similar_user_recs = ratings_df[(ratings_df["userId"].isin(similar_users)) & (ratings_df["rating"] > 4)]["movieId"]
    
    # Only use the rating that is over 10 %
    similar_user_recs = similar_user_recs.value_counts() / len(similar_users)
    similar_user_recs = similar_user_recs[similar_user_recs > .10]
    
    # Find anyone who has rated the movies in similar_user_recs 5 star
    all_users = ratings_df[(ratings_df["movieId"].isin(similar_user_recs.index)) & (ratings_df["rating"] > 4)]
    # Find the percentage of all_users who reccomend tghe movies in similar_user_recs
    all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())
    
    # Compare the percentage between similar_user_recs and all_user_recs
    # We want the movies that have a big difference between "similar" and "all"
    rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
    rec_percentages.columns = ["similar", "all"]
    
    # Devide "similar" by "all" to get the score and display it in ascending to have the most recommended movies on top
    rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]
    
    rec_percentages = rec_percentages.sort_values("score", ascending=False)
    return rec_percentages.head(10).merge(movies_df, left_index=True, right_on="movieId")[["score", "title", "genres"]]

In [18]:
# Widget for inputting a movie name and displaying similar movie recommendations
movie_name_input = widgets.Text(
    value='',
    description='Movie Title:',
    disabled=False
)
recommendation_list = widgets.Output()

# Callback function for handling text input changes
def on_type(data):
    with recommendation_list:
        recommendation_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            results = search(title)
            movie_id = results.iloc[0]["movieId"]
            display(find_similar_movies(movie_id))

movie_name_input.observe(on_type, names='value')

display(movie_name_input, recommendation_list)

Text(value='', description='Movie Title:')

Output()